In [1]:
from tqdm import tqdm
import datetime as dt
import pymssql
import pandas as pd
import numpy as np
import warnings
import glob 
import os

warnings.filterwarnings('ignore')

In [2]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [ШК], [Код склада], [Код товара], [Артикул поставщика], [Внутренний артикул], [ID Сайта], [Товарное направление],
[Товарная группа], [Товарный кластер], [Проба], [Размер изделия], [Тип изделия 1], [Тип изделия 2],
[Тип изделия 3], [Дизайн], [Бренд], [Гендерный признак], [Ценовая корзина], [Группа цен], [Остаток], [Цена Розн., за шт],
[Чистый вес], [Общий вес], [Цена изделия на бирке], [Скидка на изделии], [Запрет скидки], [Цена Закупки за 1 шт. (упр сс с НДС за единицу)], [Дата закупки]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'ДК', N'ПДК')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [3]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Архив_отчеты\Продажи\По дням\\'
list_of_files = glob.glob(path + '*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

sklad = pd.read_excel(latest_file, sheet_name = 'Продажи по ассортименту', header = 2)

stock['Код склада'] = stock['Код склада'].astype('float64') 

sklad = sklad[['Unnamed: 0','Торговая марка', 'Юр лицо', 'Код склада', 'Дата открытия объекта', 'Код объекта', 'Город', 'Наименование объекта']]
sklad1 = sklad[['Код склада', 'Код объекта']].drop_duplicates().reset_index(drop=True)
sklad1 = sklad1[sklad1['Код объекта'].notna()]

sklad1['Код объекта'] = sklad1['Код объекта'].astype(str)
sklad1['8'] = sklad1['Код объекта'].apply(lambda x: x[0])
sklad1 = sklad1[sklad1['8'] != '8']

stock = stock.merge(sklad1, left_on='Код склада', right_on = 'Код склада', how='inner')

stock = stock.drop(columns=['8'])

In [4]:
stock['Дата закупки'] = pd.to_datetime(stock['Дата закупки'], dayfirst=True, format='mixed')

stock['Старая закупка (до 31.12.2022)'] = np.where(
    stock['Дата закупки'] <= pd.to_datetime('2022-12-31', dayfirst=True, format='mixed'), 1, 0
)
stock['Новая закупка (с 01.01.2023)'] = np.where(
    stock['Дата закупки'] >= pd.to_datetime('2023-01-01', dayfirst=True, format='mixed'), 1, 0
)

data = stock.groupby(by = 'ID Сайта', as_index=False)[['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)']].sum()

data['Дата закупки'] = np.where(
    (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые и новые',
    np.where(
        (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] == 0), 'Новые', 
        np.where(
            (data['Новая закупка (с 01.01.2023)'] == 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые', 'Старые'
        )
    )
)

In [5]:
stock.drop(columns=['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)', 'Дата закупки'], inplace=True)
data.drop(columns=['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)'], inplace=True)
stock = stock.merge(data, how='outer', on='ID Сайта')

In [6]:
os.chdir(r'C:\\Users\Trenkin.Sergey\Desktop\sales')
extension = 'xlsx'
all_sales = [i for i in glob.glob('*.{}'.format(extension))]

sale = pd.DataFrame()
for name in tqdm(all_sales):
    sales = pd.read_excel(name)
    sale = pd.concat([sales, sale], ignore_index= True)

100%|██████████| 13/13 [12:54<00:00, 59.61s/it]


In [7]:
objects_with_sales = stock['Код объекта'].astype(str).unique().tolist()
sale.drop_duplicates(inplace= True)

In [8]:
data = dt.datetime.isoweekday(dt.date.today())
date_now = dt.datetime.now().date().strftime('%Y-%m-%d')
last_monitoring = (dt.date.today() - dt.timedelta(days=90)).strftime('%Y-%m-%d')

sale = sale.query('КБК in @objects_with_sales')
sale = sale[sale['Дата'] >= last_monitoring]

In [9]:
images = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Артикул-фото 2024.06.11_.xlsx')
zapret = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\запрет Артикулы к размещению.xlsx')

In [10]:
stock.drop_duplicates(inplace = True)

In [11]:
compare_tov_id = stock[['Код товара', 'ID Сайта']]
compare_tov_id.columns = ['Тов', 'ID Сайта']

In [12]:
compare_tov_id.drop_duplicates(subset=['Тов'], inplace=True)

In [13]:
sale = sale.merge(compare_tov_id, how='left', on='Тов')
sale = sale.drop(columns=['Тов'])
sale = sale[['ID Сайта', 'Количество', 'Общая сумма']]
sale.dropna(inplace=True)
sale = sale.groupby(by = 'ID Сайта', as_index=False)[['Количество', 'Общая сумма']].sum()
sale.columns = ['ID Сайта', 'Количество продаж за последние 90 дней', 'Сумма продаж за последние 90 дней']

In [14]:
result = stock.drop_duplicates(subset=['ID Сайта'])
result = result.merge(sale, how='outer', on='ID Сайта')

In [15]:
del sale, sales

In [16]:
ids = stock['ID Сайта'].dropna().tolist()

In [17]:
images['article'] = images['article'].astype(str)
images = images.query('article in @ids')

In [18]:
images = images[['article', 'published', 'Ссылка на изображение']]
images.columns = ['ID Сайта', 'Опубликован', 'Ссылка на изображение']
result = result.merge(images, how='outer', on='ID Сайта')
# result = stock.merge(images, how='outer', on='ID Сайта')

In [19]:
result = result[result['Тип изделия 1'].str.contains('СЕРЬГИ')]
result = result.drop_duplicates(subset=['ID Сайта'])
result = result[result['ID Сайта'] != '']

In [20]:
result = result.drop(columns=['Остаток'])

In [21]:
# df = result
df = stock
df = df[df['ID Сайта'] != '']

In [22]:
sum_weight = df.groupby(by = 'ID Сайта', as_index = False)[['Чистый вес', 'Общий вес']].sum()
sum_weight = sum_weight.rename(columns = {'Чистый вес': 'Сумма по чистому весу', 'Общий вес': 'Сумма по общему весу'})

In [23]:
result = result.merge(sum_weight, how='inner', on='ID Сайта')

In [24]:
percentile_ = df.groupby(by = 'ID Сайта', as_index=False)['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'].quantile(0.8)
percentile_ = percentile_.rename(columns={'Цена Закупки за 1 шт. (упр сс с НДС за единицу)': '80-й перцентиль по себестоимости'})

In [25]:
percentile_

,ID Сайта,80-й перцентиль по себестоимости
0,1000054,2309.980
1,1000091,4295.000
2,1000339,8135.730
3,1000450,218357.440
4,1000593,3372.440
...,...,...
28867,8380055,6384.892
28868,8380061,5253.316
28869,8380063,5602.332
28870,8380071,5440.430


In [26]:
result = result.merge(percentile_, how='inner', on='ID Сайта')

In [27]:
result

,ШК,Код склада,Код товара,Артикул поставщика,Внутренний артикул,ID Сайта,Товарное направление,Товарная группа,Товарный кластер,Проба,...,Цена Закупки за 1 шт. (упр сс с НДС за единицу),Код объекта,Дата закупки,Количество продаж за последние 90 дней,Сумма продаж за последние 90 дней,Опубликован,Ссылка на изображение,Сумма по чистому весу,Сумма по общему весу,80-й перцентиль по себестоимости
0,2078601593669,4486.0,ТОВ1020894,5-2310-103И2-2К,5-2310-103И2-2К_ДК,3818850,ДК,ДК СЕРЬГИ,ДК_2089,585,...,4892.51,6-04-078,Старые,16.0,211158.60,True,https://static2.585.cloud/media/products/e6f80...,93.735,94.45,4787.610
1,2078601479120,4486.0,ТОВ1091527,21-9231-17-02,21-9231-17-02_ДК,8167160,ДК,ДК СЕРЬГИ,ДК_2090,585,...,9790.77,6-04-078,Старые и новые,47.0,1039550.00,True,https://static2.585.cloud/media/products/680e7...,611.951,616.21,9790.810
2,2078601483166,4486.0,ТОВ1092827,051132621,051132621_ДК,7300972,ДК,ДК СЕРЬГИ,ДК_2094,585,...,13880.63,6-04-078,Старые,29.0,981250.00,True,https://static2.585.cloud/media/products/8a024...,930.727,941.09,13946.820
3,2078601591941,4486.0,ТОВ1092830,051132921,051132921_ДК,7301406,ДК,ДК СЕРЬГИ,ДК_2093,585,...,11384.83,6-04-078,Старые,17.0,458716.12,True,https://static2.585.cloud/media/products/7a8ea...,1084.265,1096.18,11691.992
4,2078607924873,4486.0,ТОВ1094699,12-02-0101-25957#ТОП,12-02-0101-25957#ТОП_ПДК,7936983,ПДК,ПДК СЕРЬГИ,ДК_2540,585,...,4255.54,6-04-078,Новые,54.0,607613.65,True,https://static2.585.cloud/media/products/016bf...,384.974,547.34,4206.820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9893,2078606461988,4443.0,ТОВ1307554,2424365-А500ДЧ-432,2424365-А500ДЧ-432_ДК,1106528,ДК,ДК СЕРЬГИ,ДК_2113,585,...,29361.00,2-09-083,Новые,NaN,NaN,True,https://static2.585.cloud/media/products/9b5ad...,4.342,4.50,29361.000
9894,2078424579820,4745.0,ТОВ1105816,US34280,US34280_ДК,1634925,ДК,ДК СЕРЬГИ,ДК_2094,585,...,25499.40,2-11-002,Старые,NaN,NaN,NaN,NaN,3.570,3.57,25499.400
9895,2078584570552,4745.0,ТОВ1245663,С122-4418ГР,С122-4418ГР_ПДК,1715277,ПДК,ПДК СЕРЬГИ,ДК_2544,585,...,9559.04,2-11-002,Старые,NaN,NaN,NaN,NaN,3.380,3.84,9559.040
9896,2078584550196,4745.0,ТОВ1245691,С124-6050ТР,С124-6050ТР_ПДК,1992170,ПДК,ПДК СЕРЬГИ,ДК_2553,585,...,10252.97,2-11-002,Старые,NaN,NaN,True,https://static2.585.cloud/media/products/8235a...,3.540,4.02,10252.970


In [28]:
stock = stock.groupby(by = 'ID Сайта', as_index=False)['Остаток'].sum()
result = result.merge(stock, how='left', on='ID Сайта')

In [29]:
df['Чистый вес'] = df['Чистый вес'].astype(float)
clear_weight = df.groupby(by = 'ID Сайта', as_index=False)['Чистый вес'].agg({'min', 'median', 'max'})
clear_weight.rename(columns={'median': 'Чистый вес медиана', 'max': 'Чистый вес макс', 'min': 'Чистый вес мин'}, inplace=True)
clear_weight['Чистый вес размах, %'] = (clear_weight['Чистый вес макс'] / clear_weight['Чистый вес мин'] - 1) 
result = result.merge(clear_weight, how='left', on='ID Сайта')

In [30]:
price_birka = df.groupby(by = 'ID Сайта', as_index=False)['Цена Розн., за шт'].agg({'min', 'median', 'max'})
price_birka.rename(columns={'median': 'Цена изделия на бирке медиана', 'max': 'Цена изделия на бирке макс', 'min': 'Цена изделия на бирке мин'}, inplace=True)
result = result.merge(price_birka, how='left', on='ID Сайта')
result['Цена изделия на бирке размах, %'] = (result['Цена изделия на бирке макс'] / result['Цена изделия на бирке мин'] - 1) 

In [31]:
base_discount = df.groupby(by = 'ID Сайта', as_index=False)['Скидка на изделии'].agg({'min', 'max'})
base_discount.rename(columns={'max': 'Базовая скидка макс', 'min': 'Базовая скидка мин'}, inplace=True)
result = result.merge(base_discount, how='left', on='ID Сайта')

In [32]:
df['Цена со скидками'] = np.where(
    df['Запрет скидки'] == 'Да', df['Цена Розн., за шт'], df['Цена Розн., за шт'] * (1-df['Скидка на изделии'] / 100) * 0.8 * 0.6
)

In [33]:
price_skidki = df.groupby(by = 'ID Сайта', as_index=False)['Цена со скидками'].agg({'min', 'median', 'max'})
price_skidki.rename(columns={'median': 'Цена со скидками медиана', 'max': 'Цена со скидками макс', 'min': 'Цена со скидками мин'}, inplace=True)
price_skidki['Цена со скидками размах, %'] = (price_skidki['Цена со скидками макс'] / price_skidki['Цена со скидками мин'] - 1) 
result = result.merge(price_skidki, how='left', on='ID Сайта')

In [34]:
selfprice = df.groupby(by = 'ID Сайта', as_index=False)['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'].agg({'min', 'median', 'max'})
selfprice.rename(columns={'median': 'Себестоимость медиана', 'max': 'Себестоимость макс', 'min': 'Себестоимость мин'}, inplace=True)
selfprice['Себестоимость размах, %'] = (selfprice['Себестоимость макс'] / selfprice['Себестоимость мин'] - 1) 
result = result.merge(selfprice, how='left', on='ID Сайта')

In [35]:
df['Наценка'] = df['Цена со скидками'] / df['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'] - 1
markup = df.groupby(by = 'ID Сайта', as_index=False)['Наценка'].agg({'min', 'median', 'max'})
markup.rename(columns={'median': 'Наценка медиана', 'max': 'Наценка макс', 'min': 'Наценка мин'}, inplace=True)
result = result.merge(markup, how='left', on='ID Сайта')

In [36]:
zapret['Артикул сайта'] = zapret['Артикул сайта'].astype(str)
zapret = zapret['Артикул сайта'].tolist()

result['Запрет к размещению'] = np.nan
for i in result.index:
    if result['ID Сайта'][i] in zapret:
        result['Запрет к размещению'][i] = True
    else:
        result['Запрет к размещению'][i] = False

In [37]:
ecom = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\ecom_stock_raw\Остатки 2024.07.08.xlsx')

In [38]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\discounts_zolotoy\\'
list_of_files = glob.glob(path + '\*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

discount_ecom = pd.read_excel(latest_file)

In [39]:
discount_ecom.columns = ['article', 'Опубликован', 'discount_value_promotions_1','discount_value_promotions_2', 'discount_value_promotions_3','discount_value_promotions_4']
ecom = ecom.merge(discount_ecom, how='left', on='article')

In [40]:
def discount_index(discount):
    try:
        return 1 - discount / 100
    except:
        return 1
    
ecom.discount_value_promotions_1 = ecom.discount_value_promotions_1.apply(discount_index)

ecom['Цена со скидками Ecom'] = np.where(
    (ecom['price'] == ecom['discount_price']) & (ecom['discount_value_promotions_1'] == 1), ecom['discount_price'] * ecom['discount_value_promotions_1'], 
    np.where(
        ecom['discount_value_promotions_1'] == 0.5, ecom['discount_price'] * ecom['discount_value_promotions_1'], ecom['discount_price'] * ecom['discount_value_promotions_1'] * 0.8
    )
)

In [41]:
ecom['article'] = ecom['article'].astype(str)
ecom = ecom[['article', 'Цена со скидками Ecom']]

In [42]:
ecom = ecom.query('article in @ids')
ecom.columns = ['ID Сайта', 'Цена со скидками Ecom']
result = result.merge(ecom, how='left', on='ID Сайта')

In [43]:
df['Сумма в себестоимости на остатках'] = df['Остаток'] * df['Цена Закупки за 1 шт. (упр сс с НДС за единицу)']

In [44]:
summa_ost = df.groupby(by ='ID Сайта', as_index=False)['Сумма в себестоимости на остатках'].sum()
result = result.merge(summa_ost, how='left', on='ID Сайта')

In [45]:
df['Цена за грамм'] = df['Цена со скидками'] / df['Чистый вес']
price_per_gramm = df.groupby(by = 'ID Сайта', as_index=False)['Цена за грамм'].agg({'min', 'median', 'max'})
price_per_gramm.rename(columns={'median': 'Цена за грамм медиана', 'max': 'Цена за грамм макс', 'min': 'Цена за грамм мин'}, inplace=True)
result = result.merge(price_per_gramm, how='left', on='ID Сайта')
result['Цена за грамм размах, %'] = (result['Цена за грамм макс'] / result['Цена за грамм мин'] - 1) 

In [46]:
zapret_sk = df.groupby(by = 'ID Сайта', as_index=False)['Запрет скидки'].count()

In [47]:
yes = df[df['Запрет скидки'] == 'Да']
no = df[df['Запрет скидки'] == 'Нет']

yes = yes.groupby(by = 'ID Сайта', as_index=False)['Запрет скидки'].count()
no = no.groupby(by = 'ID Сайта', as_index=False)['Запрет скидки'].count()

yes.columns = ['ID Сайта', 'Количество с запрещенной скидкой']
no.columns = ['ID Сайта', 'Количество с разрешенной скидкой']

yes = yes.merge(no, how='outer', on='ID Сайта')

result = result.merge(yes, how='left', on='ID Сайта')

In [48]:
result = result.drop_duplicates(subset=['ID Сайта'])

In [49]:
result['Размах по весу'] = np.where(
    result['Чистый вес размах, %'] >= 0.3, '3. Большой размах (>30%)',
    np.where(
        result['Чистый вес размах, %'] >= 0.1, '2. Средний размах (10-30%)', '1. Небольшой размах (<10%)'
    )
)

result['Размах по себестоимости'] = np.where(
    result['Себестоимость размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Себестоимость размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<10%)'
    )
)

result['Размах по цене со скидками'] = np.where(
    result['Цена со скидками размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Цена со скидками размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<10%)'
    )
)

result['Размах по цене за грамм'] = np.where(
    result['Цена за грамм размах, %'] >= 0.3, '3. Большой размах (>30%)',
    np.where(
        result['Цена за грамм размах, %'] >= 0.1, '2. Средний размах (10-30%)', '1. Небольшой размах (<10%)'
    )
)

result['Размах цена изделия на бирке'] = np.where(
    result['Цена изделия на бирке размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Цена изделия на бирке размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<20%)'
    )
)

In [50]:
result['Скидка разрешена'] = result['Количество с запрещенной скидкой'] + result['Количество с разрешенной скидкой']

result['Скидка разрешена'] = np.where(
    result['Скидка разрешена'].astype(str).str.contains('nan'), 'Да или нет', 'Да и нет'
)

In [51]:
result.columns

Index(['ШК', 'Код склада', 'Код товара', 'Артикул поставщика',
       'Внутренний артикул', 'ID Сайта', 'Товарное направление',
       'Товарная группа', 'Товарный кластер', 'Проба', 'Размер изделия',
       'Тип изделия 1', 'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
       'Гендерный признак', 'Ценовая корзина', 'Группа цен',
       'Цена Розн., за шт', 'Чистый вес', 'Общий вес', 'Цена изделия на бирке',
       'Скидка на изделии', 'Запрет скидки',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Код объекта',
       'Дата закупки', 'Количество продаж за последние 90 дней',
       'Сумма продаж за последние 90 дней', 'Опубликован',
       'Ссылка на изображение', 'Сумма по чистому весу',
       'Сумма по общему весу', '80-й перцентиль по себестоимости', 'Остаток',
       'Чистый вес медиана', 'Чистый вес мин', 'Чистый вес макс',
       'Чистый вес размах, %', 'Цена изделия на бирке медиана',
       'Цена изделия на бирке мин', 'Цена изделия на бирке макс',
       '

In [52]:
result = result.drop(columns = ['ШК', 'Код склада', 'Код объекта', 'Чистый вес', 'Цена изделия на бирке', 'Размер изделия', 'Общий вес', 'Скидка на изделии', 'Запрет скидки', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)',
                                'Запрет скидки', 'Цена Розн., за шт', 'Код товара'])

In [53]:
tov = result.groupby(by = 'Товарная группа')['Остаток'].sum()
tov_sales = result.groupby(by = 'Товарная группа')['Количество продаж за последние 90 дней'].sum()

result['Доля % остатка ID отосительно остатка ТГ'] = np.nan
for i in result.index: 
    t = result['Товарная группа'][i]
    result['Доля % остатка ID отосительно остатка ТГ'][i] = result['Остаток'][i] / tov[t] 
    
    
result['Доля % относительно продаж ТГ за последние 90 дней'] = np.nan
for i in result.index: 
    t = result['Товарная группа'][i]
    result['Доля % относительно продаж ТГ за последние 90 дней'][i] = result['Количество продаж за последние 90 дней'][i] / tov_sales[t] 

In [54]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9898 entries, 0 to 9909
Data columns (total 62 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Артикул поставщика                                  9898 non-null   object 
 1   Внутренний артикул                                  9898 non-null   object 
 2   ID Сайта                                            9898 non-null   object 
 3   Товарное направление                                9898 non-null   object 
 4   Товарная группа                                     9898 non-null   object 
 5   Товарный кластер                                    9898 non-null   object 
 6   Проба                                               9898 non-null   object 
 7   Тип изделия 1                                       9898 non-null   object 
 8   Тип изделия 2                                       9898 non-null   object 
 9   Ти

In [55]:
result['Модель'] = np.nan
for i in result.index:
    if (result['Размах по весу'][i] == '1. Небольшой размах (<10%)') and (result['Чистый вес макс'][i] - result['Чистый вес мин'][i]) <= 0.2:
        result['Модель'][i] = 'Штучный'
    else:
        result['Модель'][i] = 'Весовой'

In [65]:
result['Оборачиваемость'] = result['Остаток'] / result['Количество продаж за последние 90 дней'] * 3

In [66]:
result['Оборач кат'] = np.where(
    result['Оборачиваемость'] > 24, 'от 24 мес', 
    np.where(
        result['Оборачиваемость'] > 8, '8-24 мес', 'до 8 мес'
    )
)

In [70]:
result['Вес кат'] = np.where(
    result['Чистый вес мин'] > 4, 'Вес от 4 гр', 
    np.where(
        result['Чистый вес мин'] > 1.5, 'Вес 1.5 - 4 гр', 'Вес до 1.5 гр'
    )
)

In [72]:
result['Ключ ЦП'] = result['Вес кат'] + ' ' + result['Оборач кат']

In [73]:
result['Себес за грамм (чистый вес)'] = result['Сумма в себестоимости на остатках'] / result['Сумма по чистому весу']

In [75]:
result['Ключ ЦП'].value_counts()

Ключ ЦП
Вес до 1.5 гр до 8 мес      2780
Вес 1.5 - 4 гр до 8 мес     2506
Вес до 1.5 гр 8-24 мес      2264
Вес 1.5 - 4 гр от 24 мес    2153
Вес до 1.5 гр от 24 мес     1400
Вес 1.5 - 4 гр 8-24 мес     1325
Вес от 4 гр до 8 мес         185
Вес от 4 гр от 24 мес        104
Вес от 4 гр 8-24 мес          65
Name: count, dtype: int64

In [77]:
result['Ценовая политика'] = np.where(
    result['Группа цен'] == 'ШОК_ЦЕНА', 'Шок',
    np.where(
        result['Группа цен'] == 'KVI', 'KVI', 
        np.where(
            result['Ключ ЦП'].str.contains('Вес до 1.5 гр до 8 мес|Вес 1.5 - 4 гр до 8 мес|Вес до 1.5 гр 8-24 мес'), 'Высокая ЦГ',
            np.where(
                result['Ключ ЦП'].str.contains('Вес от 4 гр до 8 мес|Вес до 1.5 гр 8-24 мес|Вес до 1.5 гр от 24 мес'), 'Средняя ЦГ', 'Низкая ЦГ'
            )
        )        
    )
)

,Артикул поставщика,Внутренний артикул,ID Сайта,Товарное направление,Товарная группа,Товарный кластер,Проба,Тип изделия 1,Тип изделия 2,Тип изделия 3,...,Скидка разрешена,Доля % остатка ID отосительно остатка ТГ,Доля % относительно продаж ТГ за последние 90 дней,Модель,Оборачиваемость,Оборач кат,Вес кат,Ключ ЦП,Себес за грамм (чистый вес),Ценовая политика
0,06.63_20,06.63_20_БК,2346363,БК,БК СЕРЬГИ,БК_1560,585,СЕРЬГИ-КОНГО,БЕЗ КАМНЕЙ,КРАСНЫЙ,...,Да или нет,0.000649,0.000634,Весовой,16.200000,8-24 мес,Вес до 1.5 гр,Вес до 1.5 гр 8-24 мес,3193.367147,Высокая ЦГ
7,23.07_15,23.07_15_БК,4018455,БК,БК СЕРЬГИ,БК_1559,585,СЕРЬГИ-КОНГО,БЕЗ КАМНЕЙ,КРАСНЫЙ,...,Да или нет,0.005267,0.005177,Весовой,16.117647,8-24 мес,Вес до 1.5 гр,Вес до 1.5 гр 8-24 мес,3251.107593,Высокая ЦГ
14,134793,134793_ИФ,2001539,ИФ,ИФ ПОДВЕС КУЛЬТ,ИФ_1077,585,ПОДВЕС КУЛЬТОВЫЙ,С ФИАНИТАМИ,КРАСНЫЙ С БЕЛЫМ,...,Да или нет,0.001876,0.000674,Весовой,64.000000,от 24 мес,Вес до 1.5 гр,Вес до 1.5 гр от 24 мес,1706.559688,Средняя ЦГ
15,14.31_20,14.31_20_БК,3834609,БК,БК СЕРЬГИ,БК_1560,585,СЕРЬГИ-КОНГО,БЕЗ КАМНЕЙ,КРАСНЫЙ,...,Да или нет,0.000798,0.000482,Весовой,26.210526,от 24 мес,Вес до 1.5 гр,Вес до 1.5 гр от 24 мес,2755.369482,Средняя ЦГ
21,ПС 007#БЕЛ,ПС 007#БЕЛ_ИФ,7637180,ИФ,ИФ СЕРЬГИ,ИФ_1050,585,СЕРЬГИ,С ФИАНИТАМИ,БЕЛЫЙ,...,Да или нет,0.000434,0.000232,Весовой,30.857143,от 24 мес,Вес до 1.5 гр,Вес до 1.5 гр от 24 мес,2253.353902,Средняя ЦГ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29627,2_1714,2_1714_БК,5659120,БК,БК СЕРЬГИ,БК_1542,585,СЕРЬГИ,БЕЗ КАМНЕЙ,КРАСНЫЙ,...,Да или нет,0.000005,NaN,Штучный,NaN,до 8 мес,Вес 1.5 - 4 гр,Вес 1.5 - 4 гр до 8 мес,1692.014563,Шок
29628,С1104786,С1104786_БК,2031922,БК,БК СЕРЬГИ,БК_1543,585,СЕРЬГИ,БЕЗ КАМНЕЙ,КРАСНЫЙ,...,Да или нет,0.000005,NaN,Штучный,NaN,до 8 мес,Вес 1.5 - 4 гр,Вес 1.5 - 4 гр до 8 мес,1583.447222,Шок
29629,11-0023-11-01,11-0023-11-01_ИФ,3571826,ИФ,ИФ КОЛЬЦА,ИФ_1008,585,КОЛЬЦО,С ФИАНИТАМИ,КРАСНЫЙ,...,Да или нет,0.000004,NaN,Штучный,NaN,до 8 мес,Вес 1.5 - 4 гр,Вес 1.5 - 4 гр до 8 мес,1648.963351,Шок
29630,230103570,230103570_ИФ,7825539,ИФ,ИФ СЕРЬГИ,ИФ_1050,585,СЕРЬГИ,С ФИАНИТАМИ,БЕЛЫЙ,...,Да или нет,0.000003,0.000017,Штучный,3.000000,до 8 мес,Вес до 1.5 гр,Вес до 1.5 гр до 8 мес,1889.666667,Высокая ЦГ


In [56]:
result.to_excel(r'C:\Users\Trenkin.Sergey\Desktop\notebooks\result_1.xlsx', index=False)

In [ ]:
# date_today = dt.datetime.today().strftime('%Y-%m-%d')

In [ ]:
# result.to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\Исследование цен\\Исследование цен в разрезе ID {date_today}.xlsx', index=False)
# for tn in result['Товарное направление'].unique().tolist():
#     result[result['Товарное направление'] == tn].\
#         to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\Исследование цен\Исследование цен для {tn} в разрезе ID {date_today}.xlsx', index=False)